# Hyperparameter Tuning of a Random Forest Model for Occupancy Prediction in an Office Building

process inspired by: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74 (accessed on 2021/01/03)

## Import Modules

In [3]:
## Data Analytics

import pandas as pd
import numpy as np
import datetime
import random


## Plots

# Import matplotlib and seaborn for plotting and use magic command for Jupyter Notebooks
import matplotlib.pyplot as plt
%matplotlib inline
# Set the style for plots
plt.style.use('fivethirtyeight')
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
#sns.set(rc={'figure.figsize':(20, 10)})
# Pydot is used for visualization
import pydot


## Machine Learning

# Skicit-learn
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Cross Validation
from sklearn.model_selection import TimeSeriesSplit
# Import the model that is used - Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier 
# Import tools needed for visualization
from sklearn.tree import export_graphviz
# Import function to calculate accuracy
from sklearn.metrics import accuracy_score
# Confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
# Precision and recall
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
# Import autocorrelation function
from statsmodels.graphics.tsaplots import plot_acf
# Import Matthews Correlation Coefficient
from sklearn.metrics import matthews_corrcoef



# CSV
import csv

## Variables

In [4]:
# a = room number
a = 'E07'
# b = number of lags as input
b = '5'
# c = number of last timestep to predict
c = 0
# t = timesteps for seperate plots
#t = [6, 12, 18, 24, 30, 36]
# d = number of timestep to predict
d = 0
# e = timestep in format 't+x' as string
e = 't'

## Functions

In [5]:
# Function 1
def predict_with_model(fitted_model, X_val, y_val, output=True):
    '''
    Function to make a prediction based on a trained model, returns metrics for classification
    
    Inputs: 
    fitted_model: On a training set fitted model to use for prediction 
    X_val (df): Dataframe with feature vectors of validation (or test) data
    y_val (df): Dataframe with true labels of validation(or test) data
    output (bool): if output should be printed or not, default is True
   
    Outputs:
    y_pred (np.array): Prediction for feature vectors of validation (or test) data of the model
    metrics_dict (dict): Dictionary with metrics for classification
    accuracy (float): Accuracy of the prediction
    precision (float): Precision of the prediction
    recall (float): Recall of the prediction
    f1 (float): F1-Score of the prediction
    mcc (float): Matthews Correlation Coefficient of the prediction
    '''  
    
    # Use the trained model to make predictions on the validation (or test) set
    y_pred = fitted_model.predict(X_val)
    
    ## Metrics for classification
    # Accuracy
    accuracy = accuracy_score(y_val, y_pred)
    # Precision
    precision = precision_score(y_val, y_pred)
    # Recall
    recall = recall_score(y_val, y_pred)
    # F1-Score
    f1 = f1_score(y_val, y_pred)
    # MCC
    mcc = matthews_corrcoef(y_val, y_pred)
    
        
    metrics_dict = {'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'mcc': mcc
                   }
    
    if output:
        print('Mean of true labels:', round(np.mean(y_val), 2))
        print('Accuracy:', accuracy)
        print('Precision:', precision)
        print('Recall:', recall)
        print('F1-Score:', f1)
        print('MCC:', mcc)
            
    return y_pred, metrics_dict, accuracy, precision, recall, f1, mcc

In [6]:
# Function 2
def cross_validation_timeseries(model, X_train, y_train, n_splits, output=True):
    '''
    Function to apply a cross validation to timeseries data based on a initialized model, returns metrics for classification
    
    Inputs: 
    model: a model initialized with the chosen hyperparameters
    X_train (np.array): Numpy array with feature vectors of the training set (that is also used for validation)
    y_train (np.array): Numpy array with labels of the training set (that is also used for validation)
    n_splits (int): number of splits used in the cross-validation 
    output (bool): if output should be printed or not, default is True
    
    Outputs:
    summary_cv_dict (dict): Dictionary with mean and standard deviation of the metrics for classification
    '''
    
    # Define the way the data is split and the number of splits used for cross validation - no default value is set
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    # Create lists to save results and calculate mean and standard deviation in the end over all iterations
    acc_val_cv = []
    precision_cv = []
    recall_cv = []
    f1_cv = []
    mcc_cv = []
    size = []
    
    for train_index, val_index in tscv.split(X_train):
        cv_train, cv_val = X_train[train_index], X_train[val_index]
        cv_train_labels, cv_val_labels = y_train[train_index], y_train[val_index]
        
        # Train model on the training data
        model.fit(cv_train, cv_train_labels)
        
        # Make predictions on the validation set
        y_pred, metrics_dict, accuracy, precision, recall, f1, mcc = predict_with_model(model, cv_val, cv_val_labels, output=False)
        
        # Save results to the lists
        acc_val_cv.append(accuracy)
        precision_cv.append(precision)
        recall_cv.append(recall)
        f1_cv.append(f1)
        mcc_cv.append(mcc)
        
        # Check if Cross Validation worked properly
        size.append((cv_train.shape, cv_val.shape))
        print("Train:", train_index, "Validation", val_index)
    
    # Calculate mean and standard deviation of the metrics to evaluate
    acc_val_mean = np.mean(acc_val_cv)
    acc_val_std = np.std(acc_val_cv, ddof=1)
    precision_mean = np.mean(precision_cv)
    precision_std = np.std(precision_cv, ddof=1)
    recall_mean = np.mean(recall_cv)
    recall_std = np.std(recall_cv, ddof=1)
    f1_mean = np.mean(f1_cv)
    f1_std = np.std(f1_cv, ddof=1)
    mcc_mean = np.mean(mcc_cv)
    mcc_std = np.std(mcc_cv, ddof=1)
    
    summary_cv_dict = {'accuracy_mean': acc_val_mean,
                      'accuracy_std': acc_val_std,
                      'precision_mean': precision_mean,
                      'precision_std': precision_std,
                      'recall_mean': recall_mean,
                      'recall_std': recall_std,
                      'f1_mean': f1_mean,
                      'f1_std': f1_std,
                      'mcc_mean': mcc_mean,
                      'mcc_std': mcc_std}

    if output:    
        print("Acc_Val: {}".format(acc_val_mean))
        print("Acc_Val_Std: {}".format(acc_val_std))
        print("Precision: {}".format(precision_mean))
        print("Precision_Std: {}".format(precision_std))
        print("Recall: {}".format(recall_mean))
        print("Recall_Std: {}".format(recall_std))
        print("F1-Score: {}".format(f1_mean))
        print("F1-Score_Std: {}".format(f1_std))
        print("MCC: {}".format(mcc_mean))
        print("MCC_Std: {}".format(mcc_std))
        
    return summary_cv_dict

In [7]:
# Function 3
def evaluate_results_cv(results_cv, score):
    '''
    Function to find the best value of the chosen score to evaluate the best hyperparameters tested on the cross validation
    
    Inputs:
    results_cv (df): Dataframe with results of the cross validations
    score (str): metric that should be used to evaluate the results
            possible options are: {'accuracy_mean', 'accuracy_std',
                                    'precision_mean', 'precision_std',
                                    'recall_mean', 'recall_std',
                                    'f1_mean', 'f1_std',
                                    'mcc_mean', 'mcc_std'}
    
    Outputs:
    max_value (float): maximum score reached of the chosen metric
    idx_max_value (int): index of the corresponding cross validation in results_cv
    '''
    
    max_value = results_cv[score].max()
    idx_max_value = results_cv[score].idxmax()
    
    print('The best ' + score + ' is reached at cross validation with index ' + str((idx_max_value)) + ' with ' + str(max_value)+'.')
    
    return max_value, idx_max_value

In [8]:
# Function 4 
def create_random_hyperparameter_rf(param_grid, n_comb):
    '''
    Function to create a random set of hyperparameter as a dict out of a given parameter grid for a random forest
    
    Inputs:
    param_grid (dict): parameter grid to choose random values from
                        needs to contain values for:
                        {n_estimators, max_features, max_depth, min_samples_split, min_samples_leaf, bootstrap, random_state}
    n_comb (int): number of how many combinations should be generated (models to try)
    
    Outputs:
    hyperparameter (list): list containing the different combinations of hyperparameter as dicts
    '''

    hyperparameter = [{'n_estimators': random.choice(param_grid['n_estimators']),
                   'max_features': random.choice(param_grid['max_features']),    
                   'max_depth': random.choice(param_grid['max_depth']),
                   'min_samples_split': random.choice(param_grid['min_samples_split']),
                   'min_samples_leaf': random.choice(param_grid['min_samples_leaf']),
                   'bootstrap': random.choice(param_grid['bootstrap']),
                    'random_state': random.choice(param_grid['random_state'])} for i in range (0, n_comb)]
    
    return hyperparameter

In [9]:
# Function 5

def hyperparameter_tuning_timeseries_cv_rf(X_train, y_train, param_grid, score, n_splits, n_comb):
    '''
    Function for estimating the best hyperparameter of a random forest based on random combinations of hyperparameters 
    validated using cross-validation with time series split
    
    Inputs:
    X_train (np.array): Numpy array with feature vectors of the training set (that is also used for validation)
    y_train (np.array): Numpy array with labels of the training set (that is also used for validation)
    param_grid (dict): grid of different values for the hyperparameter to choose randomly from
    score (str): metric that should be used to evaluate to results
            possible options are: {'accuracy_mean', 'accuracy_std',
                                    'precision_mean', 'precision_std',
                                    'recall_mean', 'recall_std',
                                    'f1_mean', 'f1_std',
                                    'mcc_mean', 'mcc_std'} 
    n_splits (int): number of splits used in the cross-validation
    n_comb (int): number of how many combinations of hyperparameter should be generated (models to try)
    
    Outputs:
    results (df): Dataframe that contains the calculated metrics and the different hyperparameter
    '''
    
    # create n_comb different combinations of hyperparameter out of the param_grid
    hyperparameter_rf = create_random_hyperparameter_rf(param_grid_rf, n_comb)

    # create base model
    rf = RandomForestClassifier()
    
    for index, hyperparameters in enumerate(hyperparameter_rf):
        
        # Instantiate model
        rf_random = rf.set_params(**hyperparameters)
        
        # time series split cross validation 
        cv = cross_validation_timeseries(rf_random, X_train, y_train, n_splits=n_splits, output=False)
        
        # Create dataframe with results
        if index == 0:
            df1 = pd.DataFrame(hyperparameters, index=['0'])
            df2 = pd.DataFrame(cv, index=['0'])
            results = pd.concat([df1, df2], axis=1)
        else:
            df1 = pd.DataFrame(hyperparameters, index=['0'])
            df2 = pd.DataFrame(cv, index=['0'])
            df3 = pd.concat([df1, df2], axis=1)
            results = results.append(df3, ignore_index=True)
        
    # Evaluate results of cross validation
    max_value, idx_max_value = evaluate_results_cv(results, score)
    
    return results
        

## Import Data

In [10]:
# Read in data and set index
raw_data = pd.read_csv("\Pre-Processing\data_E07_input_5_output_144.txt", parse_dates=True)
data = raw_data.copy()
data['DateTime'] = pd.to_datetime(data['DateTime'])
data = data.set_index('DateTime')

## Edit columns

In [11]:
# Drop columns for Second
data = data.drop('Second_0', axis = 1)

In [12]:
# Drop columns of future timestamps that should not be used as input for this model
if d==0:
    for i in range(1,145):
        v = 't+'+str(i)
        data = data.drop(v, axis = 1)
else:
    for i in range(d+1,145):
        v = 't+'+str(i)
        data = data.drop(v, axis = 1)

    for i in range(1, d):
        v = 't+'+str(i)
        data = data.drop(v, axis = 1)

In [13]:
# Add columns with year, month, day and weekday name at the end of the dataset for later use of visualization 
data['Year'] = data.index.year
data['Month'] = data.index.month
data['Day'] = data.index.day
data['Weekday Name'] = data.index.day_name()
data['Hour'] = data.index.hour
data['Minute'] = data.index.minute
data['Second'] = data.index.second

## Data Preparation

In [14]:
# Use numpy to convert to arrays
# Labels are the values we want to predict
labels = np.array(data[e])

In [15]:
# Remove the labels from the data
# axis 1 refers to the columns
data = data.drop(e, axis = 1)

In [16]:
# Saving data names for later use
data_list = list(data.columns)

In [17]:
# Save copy before transforming to numpy array
data_copy = data.copy()

In [18]:
# Convert to numpy array
data = np.array(data)

In [19]:
# Extract only the one hot encoded data
# lags should not be used in this model
feature_names = [i for i in data_list if i not in ['t-5', 't-4', 't-3', 't-2', 't-1',
                                                    'Year', 'Month','Day','Hour','Minute','Second','Weekday Name']]
indices = [data_list.index(feature_names[x]) for x in range(0,len(feature_names))]
data = data[:, indices]

## Split Dataset

In [20]:
# Split data into training and testing sets using scikit-learn 
# 25 % so that approximately the last 6 months are covered
# shuffle = False because of time series data
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = 0.25, shuffle=False) 

In [21]:
# Print shapes of data sets
print('Training Data Shape:', train_data.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Data Shape:', test_data.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Data Shape: (73137, 86)
Training Labels Shape: (73137,)
Testing Data Shape: (24380, 86)
Testing Labels Shape: (24380,)


## Hyperparameter Tuning

In [23]:
# ## Create different values for the different hyperparameter to use for the parameter grid
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 5)]
# # Number of features to consider at every split
# max_features = ['auto']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True]
# # random_state
# random_state = [42]

In [24]:
## Create different values for the different hyperparameter to use for the parameter grid
# Number of trees in random forest
n_estimators = [100, 500, 775]
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
#max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
#max_depth.append(None)
max_depth = [40, 50, 60, 70]
# Minimum number of samples required to split a node
min_samples_split = [2, 4, 5, 6, 8]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3]
# Method of selecting samples for training each tree
bootstrap = [True]
# random_state
random_state = [42]

In [25]:
# Create the parameter grid
param_grid_rf = {'n_estimators': n_estimators,
                 'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
                'random_state': random_state}

## splits = 8

In [26]:
results_hyp_splits_8_comb_5_t = hyperparameter_tuning_timeseries_cv_rf(train_data, train_labels, param_grid_rf, 'mcc_mean', n_splits=8, n_comb=5)

Train: [   0    1    2 ... 8126 8127 8128] Validation [ 8129  8130  8131 ... 16252 16253 16254]
Train: [    0     1     2 ... 16252 16253 16254] Validation [16255 16256 16257 ... 24378 24379 24380]
Train: [    0     1     2 ... 24378 24379 24380] Validation [24381 24382 24383 ... 32504 32505 32506]
Train: [    0     1     2 ... 32504 32505 32506] Validation [32507 32508 32509 ... 40630 40631 40632]
Train: [    0     1     2 ... 40630 40631 40632] Validation [40633 40634 40635 ... 48756 48757 48758]
Train: [    0     1     2 ... 48756 48757 48758] Validation [48759 48760 48761 ... 56882 56883 56884]
Train: [    0     1     2 ... 56882 56883 56884] Validation [56885 56886 56887 ... 65008 65009 65010]
Train: [    0     1     2 ... 65008 65009 65010] Validation [65011 65012 65013 ... 73134 73135 73136]
Train: [   0    1    2 ... 8126 8127 8128] Validation [ 8129  8130  8131 ... 16252 16253 16254]
Train: [    0     1     2 ... 16252 16253 16254] Validation [16255 16256 16257 ... 24378 24379

In [27]:
results_hyp_splits_8_comb_5_t

,n_estimators,max_features,max_depth,min_samples_split,min_samples_leaf,bootstrap,random_state,accuracy_mean,accuracy_std,precision_mean,precision_std,recall_mean,recall_std,f1_mean,f1_std,mcc_mean,mcc_std
0,100,auto,50,5,1,True,42,0.866970,0.042845,0.615569,0.095375,0.697681,0.225784,0.631158,0.131999,0.570135,0.145203
1,775,auto,40,6,2,True,42,0.870554,0.046214,0.627094,0.099263,0.706856,0.232326,0.640086,0.142171,0.582568,0.151947
2,500,auto,70,8,1,True,42,0.868832,0.045395,0.620606,0.098183,0.713852,0.231442,0.639999,0.135057,0.581622,0.148141
3,100,auto,60,5,1,True,42,0.867462,0.041734,0.617662,0.092970,0.697421,0.228721,0.631250,0.131289,0.571207,0.143298
4,775,auto,50,6,2,True,42,0.870431,0.046209,0.626067,0.099314,0.707295,0.228619,0.640982,0.138948,0.582478,0.150720


In [28]:
results_hyp_splits_8_comb_5_t.to_csv('results_hyp_splits_8_comb_5_t.txt')

In [32]:
results_param = results_hyp_splits_8_comb_5_t[['n_estimators', 'max_features', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'bootstrap', 'random_state']]

In [33]:
results_metrics = results_hyp_splits_8_comb_5_t[['accuracy_mean', 'accuracy_std' ,'precision_mean', 'precision_std', 'recall_mean', 'recall_std', 'f1_mean', 'f1_std', 
                   'mcc_mean', 'mcc_std']]

In [38]:
results_param['index'] = [1,2,3,4,5]

In [39]:
results_param

,n_estimators,max_features,max_depth,min_samples_split,min_samples_leaf,bootstrap,random_state,index
index,,,,,,,,
1,100,auto,50,5,1,True,42,1
2,775,auto,40,6,2,True,42,2
3,500,auto,70,8,1,True,42,3
4,100,auto,60,5,1,True,42,4
5,775,auto,50,6,2,True,42,5


In [41]:
results_metrics.set_index('index')

,accuracy_mean,accuracy_std,precision_mean,precision_std,recall_mean,recall_std,f1_mean,f1_std,mcc_mean,mcc_std
index,,,,,,,,,,
1,0.866970,0.042845,0.615569,0.095375,0.697681,0.225784,0.631158,0.131999,0.570135,0.145203
2,0.870554,0.046214,0.627094,0.099263,0.706856,0.232326,0.640086,0.142171,0.582568,0.151947
3,0.868832,0.045395,0.620606,0.098183,0.713852,0.231442,0.639999,0.135057,0.581622,0.148141
4,0.867462,0.041734,0.617662,0.092970,0.697421,0.228721,0.631250,0.131289,0.571207,0.143298
5,0.870431,0.046209,0.626067,0.099314,0.707295,0.228619,0.640982,0.138948,0.582478,0.150720


## default setting in CV

In [ ]:
# try default with cv

In [29]:
# Create the parameter grid
param_grid_rf = {'n_estimators': [100],
                 'max_features': ['auto'],
               'max_depth': [None],
               'min_samples_split': [2],
               'min_samples_leaf': [1],
               'bootstrap': bootstrap,
                'random_state': random_state}

In [30]:
# try default setting in CV with 8 splits
results_hyp_splits_8_comb_1_t_def = hyperparameter_tuning_timeseries_cv_rf(train_data, train_labels, param_grid_rf, 'mcc_mean', n_splits=8, n_comb=1)

Train: [   0    1    2 ... 8126 8127 8128] Validation [ 8129  8130  8131 ... 16252 16253 16254]
Train: [    0     1     2 ... 16252 16253 16254] Validation [16255 16256 16257 ... 24378 24379 24380]
Train: [    0     1     2 ... 24378 24379 24380] Validation [24381 24382 24383 ... 32504 32505 32506]
Train: [    0     1     2 ... 32504 32505 32506] Validation [32507 32508 32509 ... 40630 40631 40632]
Train: [    0     1     2 ... 40630 40631 40632] Validation [40633 40634 40635 ... 48756 48757 48758]
Train: [    0     1     2 ... 48756 48757 48758] Validation [48759 48760 48761 ... 56882 56883 56884]
Train: [    0     1     2 ... 56882 56883 56884] Validation [56885 56886 56887 ... 65008 65009 65010]
Train: [    0     1     2 ... 65008 65009 65010] Validation [65011 65012 65013 ... 73134 73135 73136]
The best mcc_mean is reached at cross validation with index 0 with 0.5795393327075484.


In [ ]:
results_hyp_splits_8_comb_1_t_def

In [ ]:
results_hyp_splits_8_comb_1_t_def.drop(['n_estimators', 'max_features', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'bootstrap', 'random_state'], axis=1)

## results

In [ ]:
# mcc 0.4576 in CV
best_random = RandomForestClassifier(n_estimators=775, max_depth=60, min_samples_split=2, min_samples_leaf=1, random_state=42)
best_random.fit(train_data, train_labels)

In [ ]:
predict_with_model(best_random, test_data, test_labels)

In [ ]:
base_model = RandomForestClassifier(n_estimators=100, random_state=42)
base_model.fit(train_data, train_labels)
base_accuracy = base_model.score(test_data, test_labels)

In [ ]:
base_model.score(train_data, train_labels)

In [ ]:
predict_with_model(base_model, test_data, test_labels)

In [ ]:
base_model.get_params()

In [ ]:
print(base_accuracy)

In [ ]:
best_random = RandomForestClassifier(n_estimators=233, max_features='sqrt', max_depth=76, min_samples_split=2, min_samples_leaf=2, bootstrap=True)
best_random.fit(train_data, train_labels)
random_accuracy = best_random.score(test_data, test_labels)

In [ ]:
print(random_accuracy)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))